In [76]:
import os, sys

# Get the absolute path of the notebook's directory
notebook_dir = os.getcwd()

# Navigate to the parent directory
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# Add the parent directory to sys.path so we can import modules
sys.path.append(parent_dir)

# Verify the path
print(f"Added to sys.path: {parent_dir}")

# Add it to sys.path
sys.path.append(parent_dir)

print(parent_dir)
from localityaware.module import *
from NSGA_nn.nsga import *
from FlashOperation.Refrig2DrumHeatExConstrDummy import Refrig2DrumConstraintHeatExConstDummy

Added to sys.path: /Users/panwapromtep/Desktop/Aspen
/Users/panwapromtep/Desktop/Aspen


In [77]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [78]:
import numpy as np
import pickle
import os
import time

# Define the file path for saving/loading the data
data_file = "flash_simulation_data_sim_new.pkl"

# Define parameter ranges
flash_1_range_sim = np.linspace(-15, 15, 10)
flash_2_range_sim = np.linspace(-15, 15, 10)

print("Generating new data...")
assSim = Refrig2DrumConstraintHeatExConstDummy(AspenFile="../FlashOperation/FlashOperation_HeatExchanger.bkp",
                                        wdpath="../FlashOperation",
                                        visibility=False,
                                        Penalty=1e3)

start_time = time.time()  # Start timing
data_sim = []
for flash_1 in flash_1_range_sim:
    for flash_2 in flash_2_range_sim:
        x_unflat = assSim.unflatten_params([float(flash_1), float(flash_2)])
        data_sim.append([flash_1, flash_2, assSim.run_obj(x_unflat)])

data_sim = np.array(data_sim)
data_gen_time = time.time() - start_time  # Total time to generate data
total_original_assSim_calls = len(data_sim)

Generating new data...
The cost is 450.00
The cost is 361.11
The cost is 294.44
The cost is 250.00
The cost is 227.78
The cost is 227.78
The cost is 250.00
The cost is 294.44
The cost is 361.11
The cost is 450.00
The cost is 361.11
The cost is 272.22
The cost is 205.56
The cost is 161.11
The cost is 138.89
The cost is 138.89
The cost is 161.11
The cost is 205.56
The cost is 272.22
The cost is 361.11
The cost is 294.44
The cost is 205.56
The cost is 138.89
The cost is 94.44
The cost is 72.22
The cost is 72.22
The cost is 94.44
The cost is 138.89
The cost is 205.56
The cost is 294.44
The cost is 250.00
The cost is 161.11
The cost is 94.44
The cost is 50.00
The cost is 27.78
The cost is 27.78
The cost is 50.00
The cost is 94.44
The cost is 161.11
The cost is 250.00
The cost is 227.78
The cost is 138.89
The cost is 72.22
The cost is 27.78
The cost is 5.56
The cost is 5.56
The cost is 27.78
The cost is 72.22
The cost is 138.89
The cost is 227.78
The cost is 227.78
The cost is 138.89
The cos

In [79]:
# import matplotlib.pyplot as plt
# import numpy as np
# from scipy.interpolate import griddata

# def plot_contour_surface(data, x_label, y_label, z_label, x_point=None, gradient=None):
#     """
#     Generate a high-resolution heatmap with contour lines based on the given 3D data.
#     Optionally overlays a single point (x_point) and its gradient vector on the plot.
#     """
#     # Convert data to numpy array for easier manipulation
#     data = np.array(data)
#     x, y, z = data[:, 0], data[:, 1], data[:, 2]

#     # Generate a finer grid for interpolation
#     grid_x, grid_y = np.meshgrid(
#         np.linspace(x.min(), x.max(), 200),
#         np.linspace(y.min(), y.max(), 200)
#     )
    
#     # Interpolate z values using cubic interpolation
#     grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

#     # Create the heatmap and add contour lines
#     plt.figure(figsize=(8, 6))
#     plt.imshow(grid_z, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', cmap='viridis', aspect='auto')
#     plt.colorbar(label=z_label)

#     contour_levels = np.linspace(np.nanmin(grid_z), np.nanmax(grid_z), 15)
#     plt.contour(grid_x, grid_y, grid_z, levels=contour_levels, colors='black', linewidths=0.8)
    
#     # Plot the true optimal point
#     true_optimal = np.array([-1.11, -0.5])
#     plt.scatter(true_optimal[0], true_optimal[1], color='blue', zorder=5)
#     plt.text(true_optimal[0], true_optimal[1], 'True Optimal', color='white', fontsize=10, ha='right')
    
#     # Plot the local minimum points
#     local_minima = [np.array([0.84, -0.5]), np.array([0.27, -0.5])]
#     for lm in local_minima:
#         plt.scatter(lm[0], lm[1], color='orange', zorder=5)
#         plt.text(lm[0], lm[1], 'Local Min', color='orange', fontsize=10, ha='right')
    
#     # Overlay the x point and gradient arrow if provided
#     if x_point is not None and gradient is not None:
#         # Convert the x and grad lists to NumPy arrays with float type
#         gradient = np.asarray(gradient, dtype=float) # grad is grad_pred.tolist() passed in
#         plt.quiver(
#             x_point[0], x_point[1],
#             gradient[0], gradient[1],
#             color='red', angles='xy', scale_units='xy', scale=0.1, zorder=10
#         )
#         plt.scatter(x_point[0], x_point[1], color='red', zorder=10)  # Mark the point
#         plt.text(x_point[0], x_point[1], 'x_point', color='red', fontsize=10, ha='left')
    
#     plt.xlabel(x_label)
#     plt.ylabel(y_label)
#     plt.show()

In [80]:
# # Extract columns
# x, y, z = data_sim[:, 0], data_sim[:, 1], data_sim[:, 2]

# # Generate a finer grid for interpolation
# grid_x, grid_y = np.meshgrid(
#     np.linspace(x.min(), x.max(), 200),  # Increased resolution
#     np.linspace(y.min(), y.max(), 200)
# )

# # Interpolate z values using cubic interpolation
# grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

# # Create a high-resolution heatmap with contour lines
# plt.figure(figsize=(8, 6))
# plt.imshow(grid_z, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', cmap='viridis', aspect='auto')
# plt.colorbar(label='Objective Value')

# # Add contour lines
# contour_levels = np.linspace(grid_z.min(), grid_z.max(), 15)  # Define contour levels
# plt.contour(grid_x, grid_y, grid_z, levels=contour_levels, colors='black', linewidths=0.8)

# # Add labels and title
# plt.xlabel('Flash 1')
# plt.ylabel('Flash 2')
# plt.title('Fine-Grained Heatmap with Contour Lines')

# # Show the plot
# plt.show()


In [81]:
print(len(data_sim))
scaler = TorchMinMaxScaler((-1, 1), 
                           max_vals=[15.0, 15.0],
                           min_vals=[-15.0, -15.0],
                           min_y=0,
                           max_y=1e6 ,
                           scale_y=True)
#scale data_sim 
data_sim_xscaled, data_sim_yscaled = scaler.transform(data_sim[:, :2], data_sim[:, 2])

#recombine the data
data_sim_scaled = np.column_stack([data_sim_xscaled, data_sim_yscaled])

# **Initialize Model & Datasets**
dataset = DynamicDataset(data_sim_scaled)
model = MLP(2, [20,20, 20], 1)

100


In [ ]:
out = optimize_surr_nsga(model=model,
                   dataset=dataset,
                   assSim=assSim,
                   lrs={'first':1e-3, 'others':1e-2},
                   epochs={'first':1000, 'others':200},
                   min_vals=scaler.min_x,
                   max_vals=scaler.max_x,
                   scaler=scaler,
                   device='cpu',
                   iter=10,
                   print_loss=True,
                   print_it_data=True,
                   pop_size=100,
                   n_gen = 3,
                   new_data_size=5,
                   batch_size=50
                   )

Iteration 0: Training surrogate model...
Epoch 0: Total Loss=2.1054
Epoch 50: Total Loss=0.1078
Epoch 100: Total Loss=0.0215
Epoch 150: Total Loss=0.0055
Epoch 200: Total Loss=0.0020
Epoch 250: Total Loss=0.0011
Epoch 300: Total Loss=0.0007
Epoch 350: Total Loss=0.0005
Epoch 400: Total Loss=0.0003
Epoch 450: Total Loss=0.0002
Epoch 500: Total Loss=0.0002
Epoch 550: Total Loss=0.0001
Epoch 600: Total Loss=0.0001
Epoch 650: Total Loss=0.0001
Epoch 700: Total Loss=0.0001
Epoch 750: Total Loss=0.0001
Epoch 800: Total Loss=0.0000
Epoch 850: Total Loss=0.0000
Epoch 900: Total Loss=0.0000
Epoch 950: Total Loss=0.0000
n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |      100 | -1.000001E+00 | -1.008779E+00
     2 |      200 | -1.003672E+00 | -1.008870E+00
     3 |      300 | -1.006493E+00 | -1.008871E+00
The cost is 46.15
The cost is 46.15
The cost is 43.78
The cost is 51.26
The cost is 47.92
The cost is 52.91
Iteration 0: Optimal input [ 1.7273293 -6.5700464], output 46.149176377134

In [83]:
import os
import datetime
import pickle

# Get current notebook name manually (or hardcode it here)
notebook_name = "quadratic"  # or os.path.basename(__file__).replace(".py", "")

# Get timestamp
now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Construct path: results/<notebook_name>/run_<timestamp>/
base_dir = os.path.join("results", notebook_name)
save_dir = os.path.join(base_dir, f"run_{now}")
os.makedirs(save_dir, exist_ok=True)

print(f"Results will be saved in: {save_dir}")

# Save out.pkl
out["data_gen_time"] = data_gen_time
out["total_original_assSim_calls"] = total_original_assSim_calls
with open(os.path.join(save_dir, "out.pkl"), "wb") as f:
    pickle.dump(out, f)

# Save scaler
with open(os.path.join(save_dir, "scaler.pkl"), "wb") as f:
    pickle.dump(scaler, f)

# Save dataset
with open(os.path.join(save_dir, "dataset.pkl"), "wb") as f:
    pickle.dump(dataset, f)

print(f"Saved all results in {save_dir}")


Results will be saved in: results/quadratic/run_2025-04-11_18-22-01
Saved all results in results/quadratic/run_2025-04-11_18-22-01
